<a href="https://colab.research.google.com/github/MFrogy24/Gantt-Chart/blob/main/Prueba_modelo_Sequential_V3_01ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from pathlib import Path
import glob
from google.colab import drive

In [2]:
drive.mount('\content\drive')

KeyboardInterrupt: ignored

In [3]:
# Define the paths to the subdirectories
directory_path = '/content/contentdrive/MyDrive/Data-gray-aug/Data'  # Replace with the actual path to your "data" directory
subdirectory_1 = 'Siren'
subdirectory_2 = 'NoSiren'

directory_path_test = '/content/contentdrive/MyDrive/Data-gray-aug/Data-Test'  # Replace with the actual path to your "data" directory

In [4]:
AUTOTUNE = tf.data.AUTOTUNE

dataset = tf.keras.utils.image_dataset_from_directory(directory_path,
                                                      labels='inferred',
                                                      label_mode = 'binary',
                                                      color_mode= 'grayscale',
                                                      batch_size=16,
                                                      image_size=(49,49),).repeat()

dataset = dataset.prefetch(buffer_size=AUTOTUNE)
#GTC
dataset = dataset.map(lambda x,y: (tf.image.grayscale_to_rgb(x),y), num_parallel_calls=AUTOTUNE)
dataset = dataset.map(lambda x,y: (tf.image.random_flip_left_right(x), y), num_parallel_calls=AUTOTUNE)
#dataset = dataset.map(lambda x,y: (tf.keras.preprocessing.image.random_shift(x, hrg=0, wrg=0.1,  fill_mode='reflect',), y), num_parallel_calls=AUTOTUNE)
dataset = dataset.map(lambda x,y: (tf.image.rgb_to_grayscale(x),y), num_parallel_calls=AUTOTUNE)
dataset = dataset.map(lambda x,y: (tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x),y), num_parallel_calls=AUTOTUNE)

Found 1118 files belonging to 2 classes.


In [5]:
dataset_test = tf.keras.utils.image_dataset_from_directory(directory_path_test,
                                                      labels='inferred',
                                                      label_mode = 'binary',
                                                      color_mode= 'grayscale',
                                                      batch_size=16,
                                                      image_size=(49,49),).repeat()

dataset_test = dataset_test.prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.map(lambda x,y: (tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x),y), num_parallel_calls=AUTOTUNE)

Found 100 files belonging to 2 classes.


Modelo


In [6]:
model = Sequential()

model.add(Conv2D(filters        = 64,
                 kernel_size    = (3, 3),
                 padding        = 'same',
                 activation     = 'relu',
                 input_shape    = (49,49,1)))
model.add(Conv2D(filters        = 64,
                 kernel_size    = (3, 3),
                 activation     = 'relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters        = 64,
                 kernel_size    = (3, 3),
                 padding        = 'same',
                 activation     = 'relu'))
model.add(Conv2D(filters        = 64,
                 kernel_size    = (3, 3),
                 activation     = 'relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(units=64, activation= 'relu'))
model.add(Dense(units=32, activation= 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units=1, activation= 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 49, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 64)        36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 64)        36928     
                                                        

Training

In [7]:
# Compile and train the model
model.compile(optimizer = 'adam',
              loss      = 'binary_crossentropy',
              metrics   = ['accuracy'])

In [8]:
callback =  tf.keras.callbacks.ModelCheckpoint(
              filepath= 'bestModel',
              monitor = 'accuracy',
              save_best_only = True,
          )

In [9]:
BATCH_SIZE      = 16
EPOCHS          = 20
INITIAL_EPOCH   = 0
steps_per_epoch = 1118//BATCH_SIZE
validation_steps = 100//BATCH_SIZE

In [10]:
model.fit(dataset,
          batch_size       = BATCH_SIZE,
          epochs           = EPOCHS,
          callbacks        = [callback],
          initial_epoch    = INITIAL_EPOCH,
          steps_per_epoch  = steps_per_epoch,
          verbose          = 2,
          validation_data  = dataset_test,
          validation_steps = validation_steps)

Epoch 1/20
69/69 - 183s - loss: 0.5474 - accuracy: 0.6839 - val_loss: 0.3130 - val_accuracy: 0.8750 - 183s/epoch - 3s/step
Epoch 2/20
69/69 - 3s - loss: 0.1966 - accuracy: 0.9338 - val_loss: 0.2104 - val_accuracy: 0.9271 - 3s/epoch - 39ms/step
Epoch 3/20
69/69 - 3s - loss: 0.1213 - accuracy: 0.9510 - val_loss: 0.1220 - val_accuracy: 0.9688 - 3s/epoch - 40ms/step
Epoch 4/20
69/69 - 3s - loss: 0.0926 - accuracy: 0.9655 - val_loss: 0.0989 - val_accuracy: 0.9583 - 3s/epoch - 43ms/step
Epoch 5/20
69/69 - 2s - loss: 0.0546 - accuracy: 0.9819 - val_loss: 0.1030 - val_accuracy: 0.9688 - 2s/epoch - 31ms/step
Epoch 6/20
69/69 - 2s - loss: 0.0506 - accuracy: 0.9846 - val_loss: 0.0584 - val_accuracy: 0.9896 - 2s/epoch - 35ms/step
Epoch 7/20
69/69 - 2s - loss: 0.0252 - accuracy: 0.9927 - val_loss: 0.0338 - val_accuracy: 0.9896 - 2s/epoch - 30ms/step
Epoch 8/20
69/69 - 1s - loss: 0.0253 - accuracy: 0.9918 - val_loss: 0.1059 - val_accuracy: 0.9479 - 960ms/epoch - 14ms/step
Epoch 9/20
69/69 - 1s - los

In [11]:
model.save('final_model_v3_final')